<a href="https://colab.research.google.com/github/AbhayZ1/CODSOFT/blob/main/movie_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 10.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505171 sha256=44d67c604d07c82993025b20cf1cd58bbec4603db6cf901eadbc643f6b9284ec
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [2]:
import pandas as pd
import pickle
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate

# Load the movie and rating data into Pandas DataFrames
movies_path = 'movies.csv'  # Replace with the path to your movies.csv
ratings_path = 'ratings.csv'  # Replace with the path to your ratings.csv

movies = pd.read_csv(movies_path)
ratings = pd.read_csv(ratings_path)



In [ ]:
# Inspect the data (optional)
print(movies.head())
print(ratings.head())

# Prepare the data for the Surprise library
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Use SVD (Singular Value Decomposition) for collaborative filtering
model = SVD()

# Evaluate the model using cross-validation
print("\nCross-validating the model...")
cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Train the model on the entire dataset
trainset = data.build_full_trainset()
model.fit(trainset)

# Save the trained model to a file
model_path = 'svd_model.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(model, f)
print(f"\nModel saved to {model_path}.")



   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   userId  movieId  rating   timestamp
0       1      296     5.0  1147880044
1       1      306     3.5  1147868817
2       1      307     5.0  1147868828
3       1      665     5.0  1147878820
4       1      899     3.5  1147868510

Cross-validating the model...


In [ ]:
# Load the model from the file
def load_model(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

# Recommend top N movies for a specific user
def recommend_top_n(user_id, model, movies, n=10):
    all_movie_ids = movies['movieId'].unique()
    watched_movies = ratings[ratings['userId'] == user_id]['movieId']
    unwatched_movies = set(all_movie_ids) - set(watched_movies)

    recommendations = []
    for movie_id in unwatched_movies:
        predicted_rating = model.predict(user_id, movie_id).est
        recommendations.append((movie_id, predicted_rating))

    # Sort by predicted rating in descending order
    recommendations.sort(key=lambda x: x[1], reverse=True)

    # Get top N recommendations
    top_n = recommendations[:n]

    # Map movie IDs to titles
    top_n_movies = [(movies[movies['movieId'] == movie_id]['title'].values[0], rating) for movie_id, rating in top_n]
    return top_n_movies

# Get movie recommendations based on a movie name
def recommend_based_on_movie(movie_name, model, movies, ratings, user_id=1, n=10):
    if movie_name not in movies['title'].values:
        return f"Movie '{movie_name}' not found in the dataset."

    movie_id = movies[movies['title'] == movie_name]['movieId'].values[0]
    predicted_rating = model.predict(user_id, movie_id).est

    print(f"\nPredicted rating for User {user_id} on '{movie_name}': {predicted_rating:.2f}")

    print("\nTop 10 movie recommendations:")
    recommendations = recommend_top_n(user_id, model, movies, n=n)
    for i, (title, rating) in enumerate(recommendations, 1):
        print(f"{i}. {title} (Predicted Rating: {rating:.2f})")



In [ ]:
# Main interactive functionality
def main():
    global model

    # Load the saved model
    model = load_model(model_path)
    print("Model loaded successfully.")

    while True:
        print("\nOptions:")
        print("1. Get recommendations based on a movie")
        print("2. Exit")

        choice = input("Enter your choice: ")
        if choice == '1':
            movie_name = input("Enter a movie name: ")
            user_id = int(input("Enter your user ID: "))
            recommend_based_on_movie(movie_name, model, movies, ratings, user_id=user_id)
        elif choice == '2':
            print("Exiting the program. Goodbye!")
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()


In [ ]:
import pandas as pd
import pickle
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate

# Load the movie and rating data into Pandas DataFrames
movies_path = 'movies.csv'  # Replace with the path to your movies.csv
ratings_path = 'ratings.csv'  # Replace with the path to your ratings.csv

movies = pd.read_csv(movies_path)
ratings = pd.read_csv(ratings_path)

# Inspect the data (optional)
print(movies.head())
print(ratings.head())

# Prepare the data for the Surprise library
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Use SVD (Singular Value Decomposition) for collaborative filtering
model = SVD()

# Evaluate the model using cross-validation
print("\nCross-validating the model...")
cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Train the model on the entire dataset
trainset = data.build_full_trainset()
model.fit(trainset)

# Save the trained model to a file
model_path = 'svd_model.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(model, f)
print(f"\nModel saved to {model_path}.")

# Load the model from the file
def load_model(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

# Recommend top N movies for a specific user
def recommend_top_n(user_id, model, movies, ratings, n=10):
    all_movie_ids = movies['movieId'].unique()
    watched_movies = ratings[ratings['userId'] == user_id]['movieId']
    unwatched_movies = set(all_movie_ids) - set(watched_movies)

    recommendations = []
    for movie_id in unwatched_movies:
        predicted_rating = model.predict(user_id, movie_id).est
        recommendations.append((movie_id, predicted_rating))

    # Sort by predicted rating in descending order
    recommendations.sort(key=lambda x: x[1], reverse=True)

    # Get top N recommendations
    top_n = recommendations[:n]

    # Map movie IDs to titles
    top_n_movies = [(movies[movies['movieId'] == movie_id]['title'].values[0], rating) for movie_id, rating in top_n]
    return top_n_movies

# Get movie recommendations based on a movie name
def recommend_based_on_movie(movie_name, model, movies, ratings, user_id=1, n=10):
    if movie_name not in movies['title'].values:
        print(f"Movie '{movie_name}' not found in the dataset.")
        return

    movie_id = movies[movies['title'] == movie_name]['movieId'].values[0]
    predicted_rating = model.predict(user_id, movie_id).est

    print(f"\nPredicted rating for User {user_id} on '{movie_name}': {predicted_rating:.2f}")

    print("\nTop 10 movie recommendations:")
    recommendations = recommend_top_n(user_id, model, movies, ratings, n=n)
    for i, (title, rating) in enumerate(recommendations, 1):
        print(f"{i}. {title} (Predicted Rating: {rating:.2f})")

# Main functionality (only recommend once)
def main():
    # Load the saved model
    model = load_model(model_path)
    print("Model loaded successfully.")

    # Ask for movie name and user ID
    movie_name = input("Enter a movie name: ")
    user_id = int(input("Enter your user ID: "))

    # Provide recommendations based on the movie and user
    recommend_based_on_movie(movie_name, model, movies, ratings, user_id=user_id)

if __name__ == "__main__":
    main()
